In [1]:
import json
import pandas as pd

In [22]:
def format_number(x):
    if isinstance(x, float):
        if x.is_integer():
            return int(x)  # Convert to integer if the float is a whole number
        else:
            return '{:.2f}'.format(x).rstrip('0').rstrip('.')  # Keep up to 2 decimals, remove trailing zeros
    return x

def flatten_json(json_data):
    data = []
    for circuit, circuit_value in json_data.items(): # circuit size
        for benchmark, benchmark_value in circuit_value.items(): # benchmark_name
            for layout, metrics in benchmark_value.items():
                if layout == 'init':
                    continue
                if 'lookahead_swap' not in metrics:
                    continue
                basic_swap = metrics.get('basic_swap', None)
                sabre_swap = metrics.get('sabre_swap', None)
                lookahead_swap = metrics.get('lookahead_swap', None)
# Percentage Reduction=( Swap Sabre−Swap Lookahead) / Swap Sabre * 100
                basic_swap_reduction = ((basic_swap - lookahead_swap) / basic_swap * 100) if basic_swap != 0 and basic_swap is not None and lookahead_swap is not None else None
                sabre_swap_reduction = ((sabre_swap - lookahead_swap) / sabre_swap * 100) if sabre_swap != 0 and sabre_swap is not None and lookahead_swap is not None else None
                
                basic_depth = metrics.get('basic_depth', None)
                sabre_depth = metrics.get('sabre_depth', None)
                lookahead_depth = metrics.get('lookahead_depth', None)
                basic_depth_reduction = ((basic_depth - lookahead_depth) / basic_depth * 100) if basic_depth != 0 and basic_depth is not None and lookahead_depth is not None else None
                sabre_depth_reduction = ((sabre_depth - lookahead_depth) / sabre_depth * 100) if sabre_depth != 0 and sabre_depth is not None and lookahead_depth is not None else None

                data.append({
                    "benchmark": benchmark,
                    "size": circuit, # circuit size
                    "gate": benchmark_value['init']['size'], # total gates
                    "depth": benchmark_value['init']['depth'], # circuit depth
                    "layout": layout,
                    "swap basic": basic_swap, # basic swap
                    "swap sabre": sabre_swap, # sabre swap
                    "swap lookahead": lookahead_swap, # lookahead swap
                    "swap delta b": basic_swap_reduction,
                    "swap delta s": sabre_swap_reduction, # difference between sabre and lookahead, if positive then reduced but if negative then larger than sabre
                    "depth basic": basic_depth, # basic depth
                    "depth sabre": sabre_depth, # sabre depth
                    "depth lookahead": lookahead_depth, # lookahead depth
                    "depth delta b": basic_depth_reduction,
                    "depth delta s": sabre_depth_reduction
                })
    return pd.DataFrame(data)


In [23]:
def rearrange_columns(df, first_col_name):
    """
    Reorder the columns of the DataFrame so that the specified column is first.
    
    Parameters:
    - df: The input DataFrame.
    - first_col_name: The name of the column to move to the first position.
    
    Returns:
    - A DataFrame with the specified column moved to the first position.
    """
    # Check if the specified column exists in the DataFrame
    if first_col_name not in df.columns:
        raise ValueError(f"Column '{first_col_name}' does not exist in the DataFrame.")
    
    # Create a list of columns with the specified column first
    columns = [first_col_name] + [col for col in df.columns if col != first_col_name]
    
    # Reorder the DataFrame
    df_reordered = df[columns]
    
    return df_reordered

In [4]:
def open_json_from_file():
    with open("result/benchmarking_2.json") as json_file:
        return json.load(json_file)

dict_benchmark = open_json_from_file()
print(len(dict_benchmark["5"].keys()))

15


In [5]:
a = dict_benchmark["5"]["ghz"]["full_10_2"]
a_sabre_swap = a.get("sabre_swap", None)
print(isinstance(a_sabre_swap, int))

True


$\Delta s_B$

In [44]:
df = flatten_json(dict_benchmark)
df = df.map(format_number)

def rename_columns(df):
    df.columns = [
        "benchmark",
        # "n",
        "g",
        "d",
        "layout",
        r"$s_B$", # basic swap
        r"$s_S$", # sabre swap
        r"$s_L$", # lookahead swap
        # "swap (%)",
        r"$\Delta s_B$", # swap basic reduction
        r"$\Delta s_S$", # swap sabre reduction
        r"$d_B$", # basic depth
        r"$d_S$", # sabre depth
        r"$d_L$", # lookahead depth
        # "d (%)",
        r"$\Delta d_B$",
        r"$\Delta d_S$"
    ]
    return df
# df = rename_columns(df)

print(df)

    benchmark size  gate  depth            layout swap basic swap sabre  \
0          dj    5    36     11         full_10_2          0          0   
1          dj    5    36     11          full_7_3          0          0   
2          dj    5    36     11         ring_10_2         36          3   
3          dj    5    36     11          ring_7_3         24          3   
4          dj    5    36     11          grid_9_3          9          3   
..        ...  ...   ...    ...               ...        ...        ...   
404    wstate   15   253    135  t_horizontal_5_4         63         21   
405    wstate   15   253    135    t_vertical_5_4        126         45   
406    wstate   15   253    135          ring_7_3          0         15   
407    wstate   15   253    135          full_7_3          0         12   
408    wstate   15   253    135          ring_5_4          0         48   

     swap lookahead swap delta b swap delta s depth basic depth sabre  \
0                 0       

In [45]:
def save_latex_to_file(latex, circuit_size, filename=''):
    # with open(f"table/table_{filename}_{circuit_size}.tex", "w") as file:
    with open(f"table/data_{filename}_{circuit_size}.tex", "w") as file:
        file.write(latex)

def save_excel(excel, circuit_size, filename=''):
    #  with open(f"excel/excel_{filename}_{circuit_size}.tex", "w") as file:
     with open(f"excel/data_{filename}_{circuit_size}.tex", "w") as file:
        file.write(excel)

In [46]:
def build_by_benchmark(df, circuit_size):
    df = df[df['size'] == f"{circuit_size}"]
    # df_n_5 = df_n_5.replace('_', r'\_', regex=True)
    df = df.replace('_', '\\_', regex=True)
    df = df.sort_values(by=['gate', 'benchmark'])
    df = df.drop(columns=['size'])
    df = rename_columns(df)
    latex = df.to_latex(index=False, escape=False, longtable=True, caption=f"Additional swap gates and circuit depth, n = {circuit_size}", label=f"benchmark-table-{circuit_size}")
    # latex = latex.replace('_', r'\_').replace('%', r'\%')
    save_latex_to_file(latex, circuit_size, 'benchmark')
    return df

In [47]:
import os

def build_by_benchmark_excel(df, circuit_size):
    df = df[df['size'] == f"{circuit_size}"]
    df = df.sort_values(by=['gate', 'benchmark'])
    # df = df.drop(columns=['size'])
    # df = rename_columns(df)
    # with pd.ExcelWriter("excel/benchmark_layout.xlsx", mode="a" if os.path.exists("excel/benchmark_layout.xlsx") else "w", engine="openpyxl") as writer: # change mode between a = append and w = write
    with pd.ExcelWriter("excel/data_benchmark.xlsx", mode="a" if os.path.exists("excel/data_benchmark.xlsx") else "w", engine="openpyxl") as writer: # change mode between a = append and w = write
        df.to_excel(writer, sheet_name=f"n = {circuit_size}", index=False)
    return df

In [48]:
df_benchmark_5 = build_by_benchmark(df, 5)
df_benchmark_10 = build_by_benchmark(df, 10)
df_benchmark_15 = build_by_benchmark(df, 15)

/tmp/ipykernel_6772/461077298.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace('_', '\\_', regex=True)


In [54]:
build_by_benchmark_excel(df, 5)
build_by_benchmark_excel(df, 10)
build_by_benchmark_excel(df, 15)

,benchmark,size,gate,depth,layout,swap basic,swap sabre,swap lookahead,swap delta b,swap delta s,depth basic,depth sabre,depth lookahead,depth delta b,depth delta s
292,ghz,15,17,17,full_10_2,0,6,0,nan,100,17,20,17,0,15
293,ghz,15,17,17,full_7_3,0,6,0,nan,100,17,20,17,0,15
294,ghz,15,17,17,ring_10_2,0,21,111,nan,-428.57,17,26,40,-135.29,-53.85
295,ghz,15,17,17,grid_9_3,18,9,42,-133.33,-366.67,35,20,25,28.57,-25
296,ghz,15,17,17,grid_4_5,12,18,33,-175,-83.33,29,32,25,13.79,21.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,portfoliovqe,15,2505,327,grid_9_3,3018,636,1107,63.32,-74.06,2112,835,471,77.7,43.59
322,portfoliovqe,15,2505,327,grid_4_5,5277,648,768,85.45,-18.52,2244,756,412,81.64,45.5
323,portfoliovqe,15,2505,327,line_5_4,8190,891,948,88.42,-6.4,2297,695,378,83.54,45.61
324,portfoliovqe,15,2505,327,t_horizontal_5_4,5859,975,1047,82.13,-7.38,2288,893,431,81.16,51.74


In [49]:
def build_by_layout(df, circuit_size):
    df = df[df['size'] == f"{circuit_size}"]
    df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
    df = df.replace('_', '\\_', regex=True)
    df = df.drop(columns=['size']) # remove circuit size
    df = rename_columns(df)
    df = rearrange_columns(df, 'layout')
    latex = df.to_latex(index=False, escape=False, longtable=True, caption=f"Additional swap gates and circuit depth, n = {circuit_size}", label=f"benchmark-table-{circuit_size}")
    # latex = latex.replace('_', r'\_').replace('%', r'\%')
    save_latex_to_file(latex, circuit_size, 'layout')
    return df

In [50]:
def build_by_layout_excel(df, circuit_size):
    df = df[df['size'] == f"{circuit_size}"]
    df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
    # df = df.drop(columns=['size']) # remove circuit size
    # df = rename_columns(df)
    df = rearrange_columns(df, 'layout')

    # with pd.ExcelWriter("excel/excel_layout.xlsx", mode="a" if os.path.exists("excel/benchmark_layout.xlsx") else "w", engine="openpyxl") as writer: # change mode between a = append and w = write
    with pd.ExcelWriter("excel/data_layout.xlsx", mode="a" if os.path.exists("excel/data_layout.xlsx") else "w", engine="openpyxl") as writer: # change mode between a = append and w = write
        df.to_excel(writer, sheet_name=f"n = {circuit_size}", index=False)
    return df

In [51]:
df_layout_5 = build_by_layout(df, 5)
df_layout_10 = build_by_layout(df, 10)
df_layout_15 = build_by_layout(df, 15)

/tmp/ipykernel_6772/2456750932.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
/tmp/ipykernel_6772/2456750932.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
/tmp/ipykernel_6772/2456750932.py:3: DeprecationWarning: DataFrameGr

In [53]:
build_by_layout_excel(df, 5)
build_by_layout_excel(df, 10)
build_by_layout_excel(df, 15)

/tmp/ipykernel_6772/2910238682.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
/tmp/ipykernel_6772/2910238682.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
/tmp/ipykernel_6772/2910238682.py:3: DeprecationWarning: DataFrameGr

layout      benchmark size  gate  depth  \
layout                                                                
full_10_2      292       full_10_2            ghz   15    17     17   
               282       full_10_2             dj   15   118     22   
               302       full_10_2     graphstate   15   150     29   
               389       full_10_2            vqe   15   253     31   
               399       full_10_2         wstate   15   253    135   
...                            ...            ...  ...   ...    ...   
t_vertical_5_4 380  t_vertical_5_4      su2random   15   675     81   
               358  t_vertical_5_4            qnn   15   914    158   
               318  t_vertical_5_4  portfolioqaoa   15  1260    192   
               365  t_vertical_5_4         random   15  1992    412   
               325  t_vertical_5_4   portfoliovqe   15  2505    327   

                   swap basic swap sabre  swap lookahead swap delta b  \
layout                                                                  
full_10_2      292          0          6               0          nan   
               282         66          9               9        86.36   
               302         30          6              24           20   
               389          0          6               0          nan   
               399          0         12               0          nan   
...                       ...        ...             ...          ...   
t_vertical_5_4 380       5304       1086            1098         79.3   
               358       3669        600            1077        70.65   
               318       5304        879            1440        72.85   
               365       2475       1203            1800        27.27   
               325       5304        942            1251        76.41   

                   swap delta s depth basic depth sabre  depth lookahead  \
layout                                                                     
full_10_2      292          100          17          20               17   
               282            0          95          33               29   
               302         -300          51          40               34   
               389          100          31          41               31   
               399          100         135         141              135   
...                         ...         ...         ...              ...   
t_vertical_5_4 380         -1.1        1962         658              265   
               358        -79.5        1449         509              344   
               318       -63.82        2150         641              430   
               365       -49.63        2366        1214              658   
               325        -32.8        2280         834              456   

                   depth delta b depth delta s  
layout                                          
full_10_2      292             0            15  
               282         69.47         12.12  
               302         33.33            15  
               389             0         24.39  
               399             0          4.26  
...                          ...           ...  
t_vertical_5_4 380         86.49         59.73  
               358         76.26         32.42  
               318            80         32.92  
               365         72.19          45.8  
               325            80         45.32  

[127 rows x 15 columns]

In [20]:
print("Benchmark [5]\n", df_benchmark_5.value_counts("benchmark"))
print("Benchmark [10]\n", df_benchmark_10.value_counts("benchmark"))
print("Benchmark [15]\n", df_benchmark_15.value_counts("benchmark"))

Benchmark [5]
 benchmark
dj                10
ghz               10
graphstate        10
qaoa              10
qft               10
qftentangled      10
qnn               10
random            10
vqe               10
wstate            10
portfolioqaoa      9
portfoliovqe       9
realamprandom      9
su2random          9
twolocalrandom     9
Name: count, dtype: int64
Benchmark [10]
 benchmark
dj                10
ghz               10
graphstate        10
qaoa              10
qftentangled      10
vqe               10
wstate            10
qft                9
realamprandom      9
twolocalrandom     9
portfolioqaoa      8
portfoliovqe       8
qnn                8
random             8
su2random          8
Name: count, dtype: int64
Benchmark [15]
 benchmark
dj                10
ghz               10
graphstate        10
qaoa              10
vqe               10
wstate            10
qft                8
qftentangled       8
realamprandom      8
twolocalrandom     8
portfolioqaoa      7
portfoliov

In [21]:
df_layout_15

layout      benchmark     g    d s basic s sabre  \
layout                                                                         
full_10_2      292       full_10_2            ghz    17   17       0       6   
               282       full_10_2             dj   118   22      66       9   
               302       full_10_2     graphstate   150   29      30       6   
               389       full_10_2            vqe   253   31       0       6   
               399       full_10_2         wstate   253  135       0      12   
...                            ...            ...   ...  ...     ...     ...   
t_vertical_5_4 380  t_vertical_5_4      su2random   675   81    5304    1086   
               358  t_vertical_5_4            qnn   914  158    3669     600   
               318  t_vertical_5_4  portfolioqaoa  1260  192    5304     879   
               365  t_vertical_5_4         random  1992  412    2475    1203   
               325  t_vertical_5_4   portfoliovqe  2505  327    5304     942   

                    s look d basic d swap  d look  
layout                                             
full_10_2      292       0      17     20      17  
               282       9      95     33      29  
               302      24      51     40      34  
               389       0      31     41      31  
               399       0     135    141     135  
...                    ...     ...    ...     ...  
t_vertical_5_4 380    1098    1962    658     265  
               358    1077    1449    509     344  
               318    1440    2150    641     430  
               365    1800    2366   1214     658  
               325    1251    2280    834     456  

[127 rows x 10 columns]